# Chapter 09: Creating Command-line Interfaces

## 1. unitconverter的功能介紹

#### 先import unitconverter的核心模組

In [ ]:
from unitconverter.Converter import get_table, convert_units

#### 先來看看定義了基礎物件的UnitTable.py
    unit_tables資料夾內的檔案定義了各種不同單位類型的unit_table物件，所有的這些物件都繼承了UnitTable這個class
    在class UnitTable中有下列這些method：
    * convert: 用於單位轉換
    * get_units: 回傳所有支援的單位名稱

In [ ]:
class UnitTable(object):
    """
    A class to manage conversion of units for the same quantity (e.g. length,
    volume, mass)
    """

    def __init__(self):
        self.base_unit = None
        self.to_base_unit = {}
        self.from_base_unit = {}


    def convert(self, source, dest, value):
        """
        Converts a value in one unit to another.

        @param source The source unit
        @param dest The destination unit
        @param value The value to convert
        @return The value in the destination unit
        """

        if not (self.can_convert(source) and self.can_convert(dest)):
            raise ValueError('Cannot convert given units')

        if source != self.base_unit:
            value = self.to_base_unit[source](value)

        if dest != self.base_unit:
            value = self.from_base_unit[dest](value)

        return value


    def get_units(self):
        """
        Returns a list of units that this table can convert.

        @return List of valid units
        """

        convertable_units = set(self.to_base_unit.keys()) & set(self.from_base_unit.keys())
        convertable_units.add(self.base_unit)
        return convertable_units

#### 顯示time的unit_table中可以進行轉換的所有單位

In [ ]:
get_table('time').get_units()

#### 再来看看unitconverter的核心程式Converter.py
    * get_table: 回傳一個table_name所指定的unit_table物件
    * convert_units: 把用戶指定的數值轉換為以指定單位表示的形式

In [ ]:
import importlib


def get_table(table_name):
    """
    Returns an instance of a unit table given the name of the table.

    @param table_name Name of table to retrieve
    @return UnitTable instance
    """

    converter_module = importlib.import_module('unitconverter.unit_tables.%s' % table_name)
    converter_class = getattr(converter_module, table_name)()
    return converter_class


def convert_units(table_name, value, value_unit, targets):
    """
    Converts a given value in a unit to a set of target units.

    @param table_name Name of table units are contained in
    @param value Value to convert
    @param value_unit Unit value is currently in
    @param targets List of units to convert to
    @return List of conversions
    """

    table = get_table(table_name)

    results = list()
    for target in targets:
        result = {'dest_unit': target}
        try:
            result['converted_value'] = table.convert(value_unit, target, value)
        except ValueError:
            continue
        results.append(result)

    return results

#### 將200 wh 的能量分別轉換為 j, hph的形式

In [ ]:
convert_units('energy', 200, 'wh', ['j', 'hph'])

## 2. 撰寫unitconverter的Command-line Interface程式

#### Command-line Interface(CLI)的設定全部都包含在CLI.py中，CLI指令供分為兩層
    * 第一層用於選擇所需要的unit_table的類型
    * 第二層用於選擇list模式或conversion模式
        - list模式用於顯示選定的unit_table可以轉換的所有單位的名稱
        - conversion模式用於單位的轉換

In [ ]:
# import required libraries
import argparse
import inspect
from unitconverter.Converter import get_table, convert_units

### 2.1 借助python內建的setup功能，可以方便地構建CLI環境。setup功能全部位於setup.py
    setup.py定義了以下內容：
    * entry_points: 我們可以在CLI界面中輸入"unitconvert"語句來啟動這個程式
    * packages: 需要包含進來的所有程式
    注意：下面的程式無法直接執行

In [ ]:
from setuptools import setup

setup(
    name='unitconverter',
    version='0.1.0',
    entry_points={
        'console_scripts': ['unitconvert=unitconverter.CLI:run_cli'],
    },
    description='Command line tool for unit conversion',
    classifiers=[
        'Natural Language :: English',
        'Programming Language :: Python :: 2.7',
    ],
    author='Dan Nixon',
    packages=['unitconverter', 'unitconverter.unit_tables'],
    include_package_data=True, #  to include any data files in package
    zip_safe=False)

#### 先使用"python setup.py install"，安裝完以後方可以啟用命令行工具

In [ ]:
! python setup.py install

### 2.2 主幹parser 
#### 在CLI.py的run_cli方法中，先宣告一個parser用於儲存table的類別

In [ ]:
from unitconverter.Converter import get_table, convert_units
import inspect

parser = argparse.ArgumentParser(description='Tool for converting units')

parser.add_argument(
    'table',
    metavar='TABLE',
    action='store',
    type=str,
    help='Unit table to use in conversion'
)

#### 另外定義list和convert兩種功能，依靠關鍵字"list"和"convert"以在兩者之間切換

In [ ]:
subparsers = parser.add_subparsers(help='operation to be performed')

list_table_parser = subparsers.add_parser('list')
list_table_parser.set_defaults(which='list')

conversion_parser = subparsers.add_parser('convert')
conversion_parser.set_defaults(which='convert')

#### 變數props儲存用戶輸入的功能關鍵字，利用它來進行功能切換

In [ ]:
props = parser.parse_args()

if props.which == 'list':
    _run_unit_list(props)
elif props.which == 'convert':
    _run_conversion(props)

##### 查看幫助

In [ ]:
!unitconvert -h

In [ ]:
!unitconvert energy -h

In [ ]:
!unitconvert TABLE -h

In [ ]:
!unitconvert something -h

### 2.3 list功能
#### 對於list功能，新增一個argument -m，讓程式可以輸出不同單位之間的轉換關係

In [ ]:
list_table_parser.add_argument(
        '-m', '--method',
        action='store_true',
        help='Also output the conversion method from the base unit'
    )

##### 查看幫助

In [ ]:
!unitconvert energy list -h

##### 查看energy table中所有的單位

In [ ]:
!unitconvert energy list

##### 查看energy table中單位之間的轉換關係

In [ ]:
!unitconvert energy list -m

#### list功能本質上是依靠_run_unit_list方法來實現的
    *呼叫Converter.py中的get_table方法
    *當沒有argument -m傳入時，props.method為False，此時僅輸出各單位名稱
    *當有argument -m傳入時，props.method為True，此時輸出各個單位間的轉換關係

In [ ]:
def _run_unit_list(props):
    """
    Runs the command line interface for unit listing mode.

    @param props Properties parsed by argparse
    """

    table = get_table(props.table)
    print('Unit table %s can convert between the units:')
    for unit in table.get_units():
        if props.method:
            if unit == table.base_unit:
                formula = 'base unit'
            else:
                conversion = inspect.getsource(table.from_base_unit[unit])
                formula = conversion[conversion.index(':') + 1:conversion.index('\n')].strip()
            print('%s (%s)' % (unit, formula))
        else:
            print(unit)

##### 仔細看看_run_unit_list方法的實現細節

In [ ]:
table = get_table('energy')
unit = 'ev'
table.from_base_unit[unit]

In [ ]:
conversion = inspect.getsource(table.from_base_unit[unit])
conversion

In [ ]:
conversion[conversion.index(':') + 1:conversion.index('\n')]

In [ ]:
conversion[conversion.index(':') + 1:conversion.index('\n')].strip()

### 2.4 convert功能
#### 對於convert功能，新增arguments，用來儲存單位轉換中的相關數值

In [ ]:
conversion_parser.add_argument(
    'value',
    metavar='VALUE',
    type=float,
    action='store',
    help='The value to convert'
)

conversion_parser.add_argument(
    'from_unit',
    metavar='FROM',
    action='store',
    type=str,
    help='Unit to convert from'
)

conversion_parser.add_argument(
    'to_units',
    metavar='TO',
    action='store',
    nargs='+',
    type=str,
    help='Unit(s) to convert to'
)

##### 查看幫助

In [ ]:
!unitconvert energy convert -h

##### 類似convert_units('energy', 200, 'wh', ['j', 'hph'])的用法，使用CLI進行單位轉換
    注意順序

In [ ]:
! unitconvert energy convert 2500 kcal cal j ev

#### convert功能本質上是依靠_run_conversion方法來實現的
    *呼叫Converter.py中的convert_units方法

In [ ]:
def _run_conversion(props):
    """
    Runs the command line interface for unit conversion mode.

    @param props Properties parsed by argparse
    """

    results = convert_units(table_name=props.table,
                            value=props.value,
                            value_unit=props.from_unit,
                            targets=props.to_units)

    for result in results:
        print('%f %s = %f %s' % (props.value, props.from_unit,
                                 result['converted_value'],
                                 result['dest_unit']))